# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [4]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [5]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os, dotenv
import openai

dotenv.load_dotenv()

True

In [ ]:
import os
from openai import AzureOpenAI  # Make sure you're using the correct import

# Configuration settings (only need to set these once)
deployment_name = "gpt-4o-mini"
azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']  # Make sure these environment variables are set
api_key = os.environ['AZURE_OPENAI_API_KEY']
api_version = "2024-07-01-preview"  # Verify this is the correct API version for your resource

# Create the client instance (only need to do this once)
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)
print(api_key)

## 1.0 Create the system message for ChatGPT

In [7]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## 2.0 Define helper functions



In [8]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [9]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model) # use the same model as the one used for the messages
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [44]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500,temperature=0.0):
    response = client.chat.completions.create(
          model=model_name,
          messages = messages,
          max_tokens=max_response_tokens,
          temperature=temperature
                   
    )
    return response.choices[0].message.content

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [11]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [12]:
# Create the list of messages. role can be either "user" or "assistant" 
messages= [{"role": "user", "content": user_message}]

In [13]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 25


In [14]:
max_response_tokens = 100

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
### The Impact of AI on the Future of Work

As we stand on the brink of a technological revolution, the impact of artificial intelligence (AI) on the future of work is becoming increasingly apparent. From reshaping industries to changing job roles and even redefining the skills needed in the workforce, AI is set to transform the way we work in profound ways.

#### The Automation of Routine Tasks

One of the most significant impacts of AI in the workplace is the automation of routine and repetitive tasks. Many



## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [17]:
overall_max_tokens = 1096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [19]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name, max_response_tokens)

#Append assistance response 
messages.append({"role": "assistant", "content": response})
print_conversation(messages)

Token count: 175
[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
### The Impact of AI on the Future of Work

As we stand on the brink of a technological revolution, the impact of artificial intelligence (AI) on the future of work is becoming increasingly apparent. From reshaping industries to changing job roles and even redefining the skills needed in the workforce, AI is set to transform the way we work in profound ways.

#### The Automation of Routine Tasks

One of the most significant impacts of AI in the workplace is the automation of routine and repetitive tasks. Many

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
### The Impact of AI on the Future of Work: Insights for Tech Industry Leaders

As business leaders in the tech industry, you are at the forefront of innov

### Option 2: Keep the conversation within a given number of turns

In [25]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [30]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name, max_response_tokens)

#Append assistant response
messages.append({"role": "assistant", "content": response})
print_conversation(messages)


[USER]
Make the post about generative AI aimed at business leaders who have some knowledge of the technology.

[USER]
Make the post about generative AI aimed at business leaders who have some knowledge of the technology.

[USER]
Make the post about generative AI aimed at business leaders who have some knowledge of the technology.

[USER]
Make the post about generative AI aimed at business leaders who have some knowledge of the technology.

[ASSISTANT]
### Harnessing the Power of Generative AI: What Business Leaders Need to Know

As business leaders, you are no strangers to the transformative potential of technology. Generative AI, in particular, is a frontier that promises to reshape industries, streamline operations, and drive innovation. With some foundational knowledge of the technology, now is the time to delve deeper into how generative AI can create tangible value for your organization.

#### Understanding Generative AI

At its core, generative AI refers to algorithms that

[USER

## Ejercicio 1: Modificar el Mensaje del Sistema
Enunciado:
Crea un mensaje del sistema que defina a ChatGPT como un "analista financiero experto". Imprime el mensaje del sistema resultante y utilízalo en un mensaje inicial del usuario que pregunte:
"¿Cuáles son las tendencias actuales en el mercado de acciones tecnológicas?".


In [31]:

# Mensaje del sistema
base_system_message = "You are a financial analyst expert."
system_message = f"{base_system_message.strip()}"
print(system_message)

# Crear lista de mensajes
user_message = "que vision de futuro ves a las cryptomonedas" 
messages = [{"role": "user", "content": user_message}]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, max_response_tokens)

messages.append({"role": "assistant", "content": response})
print_conversation(messages)


You are a financial analyst expert.
[USER]
que vision de futuro ves a las cryptomonedas

[ASSISTANT]
La visión de futuro para las criptomonedas es compleja y multifacética. Aquí hay algunos aspectos clave que podrían definir su evolución:

1. **Adopción Generalizada**: Es probable que las criptomonedas continúen ganando aceptación tanto por individuos como por instituciones. Esto podría incluir su uso como medio de intercambio, reserva de valor y en contratos inteligentes.

2. **Regulación**: A medida que la industria madura, los gobiernos y las autoridades financieras probablemente aumentarán la



## Ejercicio 2: Contar Tokens en una Conversación Extendida
Enunciado:
Dado un conjunto inicial de mensajes, agrega tres mensajes adicionales:

"Explain the impact of inflation on stock prices."
"What are the top stocks to watch in 2025?"
"Summarize the latest market news in 50 words."
Calcula y muestra el número total de tokens tras añadir cada mensaje.


In [32]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a helpful assistant specialized in financial topics."},
    {"role": "user", "content": "What are the current trends in the tech stock market?"}
]

# Mensajes adicionales
additional_messages = [
    "Explain the impact of inflation on stock prices.",
    "What are the top stocks to watch in 2025?",
    "Summarize the latest market news in 50 words."
]

# Agregar mensajes y calcular tokens
for msg in additional_messages:
    messages.append({"role": "user", "content": msg})
    token_count = num_tokens_from_messages(messages)
    print(f"Token count after adding '{msg}': {token_count}")


Token count after adding 'Explain the impact of inflation on stock prices.': 48
Token count after adding 'What are the top stocks to watch in 2025?': 65
Token count after adding 'Summarize the latest market news in 50 words.': 82


## Ejercicio 3: Personalizar el Mensaje del Sistema Según el Usuario
Enunciado:
Escribe un programa que permita al usuario definir el rol de ChatGPT mediante una entrada de texto (input). Una vez definido, responde al mensaje:
"What are your key skills as an assistant?"
Usa el mensaje del sistema definido por el usuario.


In [ ]:

# Entrada del usuario para definir el mensaje del sistema
user_defined_role = input("Define the role of the assistant: ")
system_message = f"You are {user_defined_role.strip()}." #Se puede escribir el mensaje arriba del visual

# Crear lista de mensajes
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "What are your key skills as an assistant?"}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, max_response_tokens)

messages.append({"role": "assistant", "content": response})

# Mostrar la conversación
print_conversation(messages)


[SYSTEM]
You are eres un asistente de matematicas.

[USER]
What are your key skills as an assistant?

[ASSISTANT]
As an assistant focused on mathematics, my key skills include:

1. **Problem Solving**: I can help you solve a wide range of mathematical problems, from basic arithmetic to advanced calculus and statistics.

2. **Concept Explanation**: I can explain mathematical concepts and theories in a clear and understandable manner.

3. **Step-by-Step Guidance**: I can provide step-by-step solutions to problems, helping you understand the process involved.

4. **Formula Application**: I can assist with the



## Ejercicio 4: Crear un Generador de Respuestas Creativo
Enunciado:
Diseña un programa que permita configurar la creatividad del modelo (temperatura) a través de una entrada del usuario. Una vez configurado, responde a la pregunta:
"Can you suggest an out-of-the-box idea for a startup?"



In [47]:

# Entrada del usuario para definir la creatividad
temperature = float(input("Set the creativity level (0.0 - 1.0): "))
if not (0.0 <= temperature <= 1.0):
    raise ValueError("Creativity level must be between 0.0 and 1.0.")

# Crear lista de mensajes
system_message = "You are a highly creative assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "me puedes dar una idea de startup"}
]

# Enviar mensaje con creatividad ajustada
response = send_message(messages, deployment_name, max_response_tokens, temperature)

#Mostrar la respuesta
print(response)

¡Claro! Aquí tienes una idea de startup:

**Plataforma de Intercambio de Habilidades Locales**

**Descripción:** Crea una plataforma que conecte a personas en una comunidad local que deseen intercambiar habilidades. Por ejemplo, alguien que sepa tocar la guitarra podría ofrecer clases a cambio de lecciones de cocina de otra persona. La idea es fomentar el aprendizaje y la colaboración entre vecinos, promoviendo un sentido de comunidad y reduciendo la necesidad de dinero en transacciones.




## Ejercicio 5: Cambiar el Tono del Mensaje
Enunciado:
Configura el mensaje del sistema para que ChatGPT responda en un tono humorístico y escribe una respuesta al mensaje:
"Explain quantum computing in a funny way."


In [39]:

# Mensaje del sistema humorístico
system_message = "You are a humorous assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Explain quantum computing in a funny way."}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, max_response_tokens)

#append assistant response
messages.append ({"role": "assistant", "content": response})
print_conversation(messages)


[SYSTEM]
You are a humorous assistant.

[USER]
Explain quantum computing in a funny way.

[ASSISTANT]
Sure! Imagine you're trying to find your car keys. In the classical world, you’re like an annoyed detective who methodically searches every inch of your house, checking under couch cushions and inside the fridge (because, let’s be honest, you’re hungry and it’s possible they fell in there during a snack break).

Now, enter the quantum world! It's like you suddenly have access to a superpower where you can actually be in two places at once. You could simultaneously look in the fridge and



## Ejercicio 6: Resumir Mensajes Previos
Enunciado:
Dado un historial de conversación de 5 mensajes, crea una función que resuma el contenido de los mensajes del usuario en una sola oración.



In [40]:

# Historial de mensajes
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me about AI."},
    {"role": "user", "content": "How does machine learning work?"},
    {"role": "user", "content": "Can you explain neural networks?"},
    {"role": "user", "content": "What are the applications of deep learning?"}
]

# Resumir mensajes del usuario
def summarize_user_messages(messages):
    user_messages = [msg['content'] for msg in messages if msg['role'] == 'user']
    summary_prompt = "Summarize the following messages into one sentence: " + " ".join(user_messages)
    summary = send_message([{"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": summary_prompt}], deployment_name)
    return summary

# Obtener y mostrar el resumen
summary = summarize_user_messages(messages)
print(f"User messages summary: {summary}")


User messages summary: AI encompasses machine learning, which uses algorithms to enable computers to learn from data; neural networks, a key architecture in this field, mimic the way the human brain works; and deep learning, a subset of machine learning, has numerous applications including image recognition, natural language processing, and autonomous systems.


## Ejercicio 7: Configuración de Temperatura y Tono
Enunciado:
Escribe un programa que cambie la configuración de temperatura a 0.9 para obtener respuestas más creativas y responde al mensaje:
"Suggest a unique title for a blog about AI in education."



In [45]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a creative assistant."},
    {"role": "user", "content": "Suggest a unique title for a blog about AI in education."}
]

# Ajustar temperatura y enviar mensaje
response = send_message(messages, deployment_name, max_response_tokens,temperature=0.9)

# Mostrar la respuesta
print(response)


"EduTech Evolution: Harnessing AI for Tomorrow's Classrooms"
